<a href="https://colab.research.google.com/github/roquearmas/IABD_DATA/blob/main/Predicci%C3%B3n_y_salida_de_Reconocimiento_Acordes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción y salida del reconocimiento de acordes

Este cuaderno forma parte del TFM de Jesús Roque Armas Martín sobre reconocimiento de acordes con redes convolucionales.

Predice los acordes de una canción dada, mediante un modelo preentrenado en otro cuaderno.

Importación de las librerías necesarias

In [1]:
import librosa
import librosa.display

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from google.colab import drive
import itertools
import math
from IPython.display import Audio
import math

import tempfile
import os
import keras.models

import cv2
from PIL import Image


Librosa(https://zenodo.org/badge/6309729.svg)


Carga del fichero de audio a predecir

In [46]:
drive.mount('/content/drive')
# audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/Dover - King George Acústico 2012.mp3'
audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/02 - Eric Clapton - Before You Accuse Me .mp3'
#audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/Lynyrd Skynyrd - 01 - Sweet Home Alabama.mp3'
# Carga en la librería Librosa
audio_data, sample_rate = librosa.load(audio_file)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Carga del modelo preentrenado

In [3]:
model = keras.models.load_model('/content/drive/MyDrive/proyecto deteccion acordes/modelo_entrenado/recon_acordes.h5')


Array de acordes soportados, en el mismo orden en que se entrenó el modelo

In [4]:
lista_acordes= ['Am', 'Bb', 'C', 'Dm', 'Bdim', 'G', 'F', 'Em']
acordes =np.array(lista_acordes)

**Tratamiento del fichero de audio**:

1.   Extraer los datos del fichero de audio, de la misma manera que cuando se entrenó el modelo

    -   División del fichero en muestras de medio segundo
    -   Normalización de la muestra a imagen en blanco y negro
    -   Reescalado de la imagen a 300 x 300 píxeles

2.   Predecir cada muestra de tiempo de la canción
3.   Generar un fichero de salida con el tiempo de inicio y fin de la muestra y el acorde predicho, así como el porcentaje de confianza que tiene en dicha predicción



In [48]:
# Divide el audio en segmentos de 1/2 segundo y genera histogramas y periodogramas
segment_duration = 0.5  # Duración del segmento en segundos
num_segments = int(len(audio_data) / (sample_rate * segment_duration))
#separar la parte armónica de la percusiva del audio de origen
audio_harmonic, audio_percussive = librosa.effects.hpss(audio_data)
salida = []
for i in range(num_segments):

    start_time = i * segment_duration
    end_time = (i + 1) * segment_duration
    segment = audio_harmonic[int(start_time * sample_rate):int(end_time * sample_rate)]

    # Deshabilitar la visualización interactiva de las imágenes
    plt.ioff()

    # Periodograma del segmento
    plt.figure(figsize=(4, 4))
    librosa.display.specshow(librosa.power_to_db(abs(librosa.stft(segment)), ref=np.max), y_axis='log', x_axis='time', cmap='viridis')
    plt.axis('off')
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
     # Guardar la imagen
    # nombreimagen = os.path.join('/content/', 'imagen.png')
    # plt.savefig(nombreimagen, bbox_inches='tight', pad_inches=0)
    # Convertir la figura a un array de NumPy
    fig = plt.gcf()
    fig.canvas.draw()
    espectrograma_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    espectrograma_array = espectrograma_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    # Cerrar la figura de Matplotlib
    plt.close()
    imagen=normalizar_imagen(espectrograma_array)
    # normalizar_imagen('/content/')
    #cargar imagen en X
    X = []
    X.append(imagen)
      #hacer predict con el modelo
    pred = model.predict(np.array(X))
    #añadir linea con el predict a un fichero de texto
    row = []
    if round(np.max(pred)*100,2)>50:
      row.append(start_time) # inicio del segmento a estudiar
      row.append(end_time-0.01)   # fin del segmento
      predicho = np.argmax(pred) #clase con mejor predicción
      row.append(acordes[predicho]) #acorde que pertenece a esa clase
      row.append(round(np.max(pred)*100,2))
      salida.append (row) #añado la fila al array de la canción
      print(row)

# Escribir los datos en el archivo de texto
with open('/content/salida.srt', 'w') as file:
    for row in salida:
        row_str = ' '.join(map(str, row))  # Convertir la fila a una cadena de texto separada por espacios
        file.write(row_str + '\n')  # Escribir la fila en el archivo



1/1 [==============================] - 0s 92ms/step
[1.0, 1.49, 'Em', 54.54]
1/1 [==============================] - 0s 91ms/step
[1.5, 1.99, 'Em', 95.26]
1/1 [==============================] - 0s 92ms/step
[2.0, 2.49, 'C', 72.13]
1/1 [==============================] - 0s 92ms/step
[2.5, 2.99, 'C', 61.41]
1/1 [==============================] - 0s 92ms/step
[3.0, 3.49, 'G', 64.33]
1/1 [==============================] - 0s 89ms/step
[8.0, 8.49, 'Em', 99.31]
1/1 [==============================] - 0s 89ms/step
[8.5, 8.99, 'Am', 67.99]
1/1 [==============================] - 0s 88ms/step
[9.0, 9.49, 'Am', 82.59]
1/1 [==============================] - 0s 89ms/step
[9.5, 9.99, 'Em', 87.55]
1/1 [==============================] - 0s 94ms/step
[10.0, 10.49, 'Am', 72.75]
1/1 [==============================] - 0s 92ms/step
[10.5, 10.99, 'C', 60.17]
1/1 [==============================] - 0s 91ms/step
[11.0, 11.49, 'Em', 61.34]
1/1 [==============================] - 0s 88ms/step
[12.0, 12.49, 'C', 88.

Paso la imagen a escala de grises, y la reescalo a 300 x 300 píxeles, antes de evaluarla

In [33]:
def normalizar_imagen(espectrograma):
  # Convertir a escala de grises
  imagen_gris = cv2.cvtColor(espectrograma, cv2.COLOR_BGR2GRAY)
  espectrograma_gris_3_canales = cv2.cvtColor(imagen_gris, cv2.COLOR_GRAY2BGR)
  #reescalar imagen
  imagen = cv2.resize(espectrograma_gris_3_canales, (300,300), interpolation=cv2.INTER_CUBIC)
  # imagen_gris = Image.fromarray(imagen_gris)
  return imagen
